# Capstone Project `The Battle of Neighborhoods`

***

## Public Restrooms in Berlin
***

### Goal
The city of Boston/MA provides a neat map with the location of publically accessible restrooms. It is available at https://www.boston.gov/departments/311/public-restrooms-city-boston:

***
See screenshot: https://github.com/u010962/Coursera_Capstone/blob/main/Bildschirmfoto%202020-12-27%20um%2014.06.55.png
***

Such information is useful for tourists with small children and older people. There seems to be no such map available to the public for Germany's capital Berlin. But fortunately, location data of public restrooms are accessible through Berlin Open Data, an initiative of the local government of Berlin.

The goal of this project is to create a map similar to the one of Boston with data of Berlin in Python. My analysis of the two cities would include a comparison of the density of restrooms in the city centers with an emphasis on facilities for physically handicapped people. I am also curious about the information provided by Foursquare, whether it is as comprehensive and complete as the one generated from publically available information.

The available dataset from Berlin's government is dated 10/08/20 and contains a name, a description, accessability by handicapped people, and position coordinates among other things. It is expected that Foursquare provides the same information, but it is not required for the purpose of this project. Most public restrooms in Berlin are operated by the company Wall GmbH.

**Import necessary libraries**

In [191]:
# -*- coding: utf-8 -*-

import requests # library to handle requests
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


***Read data from Website***

In [210]:
file = r'https://www.berlin.de/sen/uvk/_assets/verkehr/infrastruktur/oeffentliche-toiletten/berliner-toiletten-standorte.xlsx'
table = pd.read_excel(file, encoding="utf8")

# Let's get a limpse on the available data
table.head()

,Stand: 08.10.2020,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LavatoryID,Description,City,Street,Number (Time),PostalCode,Country,Longitude,Latitude,isOwnedByWall,isHandicappedAccessible,Price,canBePayedWithCoins,canBePayedInApp,canBePayedWithNFC,hasChangingTable,LabelID
3,Wall_101003,Toilette Ottmachauer Steig (oberhalb Badestell...,Berlin,"Krumme Lanke, Quermatenweg",(0-24 Uhr),14109,Deutschland,13.2406,52.4536,1,0,0,0,0,0,0,5
4,Wall_10357,"Wall CT, Ostseestr.",Berlin,Ostseestr.,1 (0-24 Uhr),10409,Deutschland,13.4305,52.5511,1,1,0.5,1,0,0,0,2


Turns out that the first three lines are useless and the fourth line contains the header. So we'll have to apply a couple of modifications to the dataset before we can feed the map with it.

1. Get rid of the first empty lines
2. Make line 2 the header of the dataframe

***
### Data Wrangling

***Read data again, with unneccessary lines omitted and proper headers***

In [211]:
df = pd.read_excel(file, skiprows=3)
df.head()

,LavatoryID,Description,City,Street,Number (Time),PostalCode,Country,Longitude,Latitude,isOwnedByWall,isHandicappedAccessible,Price,canBePayedWithCoins,canBePayedInApp,canBePayedWithNFC,hasChangingTable,LabelID
0,Wall_101003,Toilette Ottmachauer Steig (oberhalb Badestell...,Berlin,"Krumme Lanke, Quermatenweg",(0-24 Uhr),14109,Deutschland,13.240575,52.453581,1,0,0.0,0,0,0,0,5
1,Wall_10357,"Wall CT, Ostseestr.",Berlin,Ostseestr.,1 (0-24 Uhr),10409,Deutschland,13.430505,52.551083,1,1,0.5,1,0,0,0,2
2,Wall_112911,Toilette Uferweg,Berlin,"Schlachtensee , Am Schlachtensee",ggü. 145 (0-24 Uhr),14129,Deutschland,13.199229,52.436507,1,0,0.0,0,0,0,0,5
3,Wall_115753,Toilette Am Kiesteich 50,Berlin,Spektepark,(0-24 Uhr),13589,Deutschland,13.164859,52.548840,1,0,0.0,0,0,0,0,5
4,Wall_116739,"Wall CT, Lindenufer",Berlin,Lindenufer,ggü. 10 (0-24 Uhr),13597,Deutschland,13.207764,52.537351,1,1,0.5,1,1,0,0,1


***Let's analyse the data in more detail***

We want to know the size of the dataframe, the type of data, and the presence of any duplicates

In [212]:
# Size and shape
print('Size = {}\nShape ={}\n'.format(df.size, df.shape))

Size = 4386
Shape =(258, 17)



In [213]:
# Find duplicate IDs
duplicate_rows = df[df.duplicated('LavatoryID')]

if duplicate_rows.empty:
    print('No duplicate IDs found\n')

No duplicate IDs found



In [214]:
# Type of data
df.dtypes

LavatoryID                  object
Description                 object
City                        object
Street                      object
Number (Time)               object
PostalCode                   int64
Country                     object
Longitude                  float64
Latitude                   float64
isOwnedByWall                int64
isHandicappedAccessible      int64
Price                      float64
canBePayedWithCoins          int64
canBePayedInApp              int64
canBePayedWithNFC            int64
hasChangingTable             int64
LabelID                      int64
dtype: object

***OK, let's make a couple of rows boolean***

In [215]:
df['isOwnedByWall'] = df['isOwnedByWall'].astype(bool)
df['isHandicappedAccessible'] = df['isHandicappedAccessible'].astype(bool)

df['canBePayedWithCoins'] = df['canBePayedWithCoins'].astype(bool)
df['canBePayedInApp'] = df['canBePayedInApp'].astype(bool)
df['canBePayedWithNFC'] = df['canBePayedWithNFC'].astype(bool)

df['hasChangingTable'] = df['hasChangingTable'].astype(bool)

df.dtypes

LavatoryID                  object
Description                 object
City                        object
Street                      object
Number (Time)               object
PostalCode                   int64
Country                     object
Longitude                  float64
Latitude                   float64
isOwnedByWall                 bool
isHandicappedAccessible       bool
Price                      float64
canBePayedWithCoins           bool
canBePayedInApp               bool
canBePayedWithNFC             bool
hasChangingTable              bool
LabelID                      int64
dtype: object

***Changing 'Number (Time)' to 'Time' for convenience***

In [216]:
df.rename(columns={'Number (Time)':'Time'}, inplace=True)

***Check if there are any unassigned values or NaN***

In [217]:
print('Total Number of unassigned values in the entire dataframe: ', df.isnull().sum().sum())
print('Number of unassigned values in column Time: {}\n'.format(df['Time'].isnull().sum()))

Total Number of unassigned values in the entire dataframe:  17
Number of unassigned values in column Time: 17



***Replace NaN with the string 'Unknown'***

In [218]:
df['Time'].fillna(value='Unknown', inplace=True)

print('Total Number of unassigned values in the entire dataframe: ', df.isnull().sum().sum())
print('Number of unassigned values in column Time: {}\n'.format(df['Time'].isnull().sum()))

Total Number of unassigned values in the entire dataframe:  0
Number of unassigned values in column Time: 0



In [219]:
df.tail() # Take a look at the dataframe

,LavatoryID,Description,City,Street,Time,PostalCode,Country,Longitude,Latitude,isOwnedByWall,isHandicappedAccessible,Price,canBePayedWithCoins,canBePayedInApp,canBePayedWithNFC,hasChangingTable,LabelID
253,Fremd_5,Toilette Tempelhofer Feld,Berlin,Tempelhofer Feld,Unknown,12049,Deutschland,13.41755,52.47681,False,True,0.0,False,False,False,False,6
254,Fremd_6,Toilette Tempelhofer Feld,Berlin,Tempelhofer Feld,Unknown,12049,Deutschland,13.41766,52.47019,False,True,0.0,False,False,False,False,6
255,Fremd_7,Toilette Tempelhofer Feld,Berlin,Tempelhofer Feld,Unknown,12101,Deutschland,13.38677,52.47130,False,True,0.0,False,False,False,False,6
256,Fremd_8,Toilette Tempelhofer Feld,Berlin,Tempelhofer Feld,Unknown,10965,Deutschland,13.40995,52.47940,False,True,0.0,False,False,False,False,6
257,Fremd_9,Toilette Tempelhofer Feld,Berlin,Tempelhofer Feld,Unknown,10965,Deutschland,13.40423,52.48170,False,False,0.0,False,False,False,False,6


***
### Explore and cluster the locations

***Install and import folium***

In [220]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: done

# All requested packages already installed.

Folium installed and imported!


***Get the location of Berlin's center from Wikipedia : 52.518611, 13.408333 and superimpose the locations of the restrooms onto a map***

In this map we want to distinguish between restrooms that are accessible by handicapped people (outer circle is yellow) versus those that are not (outer circle is blue). Additionally, we want to get an overview on the restrooms that are accessible for free (inner circle is black) compared to those that have to be payed for (inner circle red). The zooms is set to a value that shows Berlin in its entirity - not just the city's center.

In [221]:
berlin_map = folium.Map(location=[52.518611, 13.408333], zoom_start=11)

# instantiate a feature group for the locations in the dataframe
locations = folium.map.FeatureGroup()

# loop through the restrooms and add each to the locations feature group
for lat, lng, handicappedAccessible, price in zip(df.Latitude, df.Longitude, df.isHandicappedAccessible, df.Price):
    locations.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow' if handicappedAccessible else 'blue',
            fill=True,
            fill_color='black' if price == 0 else 'red',
            fill_opacity=0.6
        )
    )

# add locations to map
berlin_map.add_child(locations)

***Group the markers into clusters and add some useful information to the labels***

This view shall (hopefully) show us an even spread of public restrooms in the city of Berlin. Individual markers are green if the respective restroom can be accessed by somebody who is physically handicapped and red otherwise. Popups shall show some useful information about the selected restrooms, e.g. opening times, price, and accessibility by wheelchair.

In [222]:
# import plugins from folium
from folium import plugins

In [239]:
# let's start with a clean copy of the map of Berlin
berlin_map = folium.Map(location=[52.518611, 13.408333], zoom_start=13)

# instantiate a mark cluster object for the locations in the dataframe
locations = plugins.MarkerCluster().add_to(berlin_map)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, handicappedAccessible, description, price, time in zip(df.Latitude, df.Longitude, df.isHandicappedAccessible, df.Description, df.Price, df.Time):
    
    header = """<meta http-equiv="content-type" content="text/html; charset=utf-8">"""
    header += "<h1>" + description + "</h1><br>"
    body = "<body>"
    body += "<pre>" + "Open: " + str(time) + "</pre>" + "<br>"
    body += "<pre>" + "Price: Euros " + str(price) + "</pre>" + "<br>"
    body += "<pre>" + ("Wheelchair-" if handicappedAccessible else "Not wheelchair-") + "accessible" + "</pre>" + "<br>"
    footer = "</body>"
    html = header + body + footer
    
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='green',icon='ok-sign') if handicappedAccessible 
        else folium.Icon(color='red',icon='exclamation-sign'),
        popup=folium.Popup(html),
    ).add_to(locations)

# display map
berlin_map

***
***Next we want to see what Foursquare has to offer in terms of public restrooms in Berlin***

Finding proper search queries might be challenging. Since 'Toilette' is the German word for 'restroom', we will go with that one.

In [294]:
# Set Foursquare access token

CLIENT_ID = 'NNOJHBNK2SQU0H2BX3P0JNKMOCTD15OIVUJ50JHSTDSHZL55' # your Foursquare ID
CLIENT_SECRET = 'PNPEL4BCMV4NST20GMPKPYZQBFYYJ1PU5KUVPPFVBTN4WNL3' # your Foursquare Secret
ACCESS_TOKEN = 'US0PSWALQJYW41UD5RURJ1GRD02TDMVMUFAMUQH5H1BUGFOD' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 250

# Using the same location for Berlin's city center as above [52.518611, 13.408333]
latitude = 52.518611
longitude = 13.408333

# Define query and search radius in meters
search_query = 'Toilette'
radius = 30000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

# Send the GET Request and examine the results
results = requests.get(url).json()

# Get relevant part of JSON and transform it into a pandas dataframe
venues = results['response']['venues']

# transform venues into a dataframe
dataframe = json_normalize(venues)

dataframe.head()

<ipython-input-294-e9ee1d51dfe4>:27: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,5d725f0b3978b20008bd818a,Toilette 2.OG Fahrstühle,"[{'id': '5744ccdfe4b0c0459246b4c4', 'name': 'P...",v-1609327072,False,Leipziger Platz 12,52.510796,13.380881,"[{'label': 'display', 'lat': 52.510796, 'lng':...",2053,10117,DE,Berlin,Berlin,Deutschland,"[Leipziger Platz 12, 10117 Berlin]",NaN
1,588a7dcf8f0be469f0e5cd35,La toilette de Marcel,"[{'id': '4bf58dd8d48988d193941735', 'name': 'W...",v-1609327072,False,NaN,52.530535,13.400316,"[{'label': 'display', 'lat': 52.530535, 'lng':...",1434,NaN,DE,NaN,NaN,Deutschland,NaN,NaN
2,57537a70498e1ca647cc24d2,City Toilette,"[{'id': '5744ccdfe4b0c0459246b4c4', 'name': 'P...",v-1609327072,False,Knaackstr. 42,52.535647,13.417005,"[{'label': 'display', 'lat': 52.53564654947588...",1985,10435,DE,Berlin,Berlin,Deutschland,"[Knaackstr. 42 (Kollwitzstr.), 10435 Berlin]",Kollwitzstr.
3,5adf817cc0363576a98b012d,City Toilette am Kottbusser Tor,"[{'id': '5744ccdfe4b0c0459246b4c4', 'name': 'P...",v-1609327072,False,NaN,52.498494,13.418281,"[{'label': 'display', 'lat': 52.498494, 'lng':...",2338,NaN,DE,NaN,NaN,Deutschland,NaN,NaN
4,579cf3e8498ea8f0308ab226,CITY TOILETTE Helmholtzplatz,"[{'id': '5744ccdfe4b0c0459246b4c4', 'name': 'P...",v-1609327072,False,Helmholtzplatz,52.542879,13.418653,"[{'label': 'display', 'lat': 52.542879, 'lng':...",2790,10437,DE,Berlin,Berlin,Deutschland,"[Helmholtzplatz, 10437 Berlin]",NaN


In [295]:
print("Number of venues found", len(dataframe.index))

Number of venues found 11


***Define information of interest and filter dataframe***

In [296]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]


# extract restrooms from the list from the list
berlin_filtered = dataframe_filtered[dataframe_filtered['name'].str.contains(search_query)]
berlin_filtered.reset_index(drop=True, inplace=True)

berlin_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Toilette 2.OG Fahrstühle,Public Bathroom,Leipziger Platz 12,52.510796,13.380881,"[{'label': 'display', 'lat': 52.510796, 'lng':...",2053,10117,DE,Berlin,Berlin,Deutschland,"[Leipziger Platz 12, 10117 Berlin]",NaN,5d725f0b3978b20008bd818a
1,City Toilette,Public Bathroom,Knaackstr. 42,52.535647,13.417005,"[{'label': 'display', 'lat': 52.53564654947588...",1985,10435,DE,Berlin,Berlin,Deutschland,"[Knaackstr. 42 (Kollwitzstr.), 10435 Berlin]",Kollwitzstr.,57537a70498e1ca647cc24d2
2,City Toilette am Kottbusser Tor,Public Bathroom,NaN,52.498494,13.418281,"[{'label': 'display', 'lat': 52.498494, 'lng':...",2338,NaN,DE,NaN,NaN,Deutschland,NaN,NaN,5adf817cc0363576a98b012d
3,Wall City-Toilette,Public Bathroom,NaN,52.520073,13.461147,"[{'label': 'display', 'lat': 52.520073, 'lng':...",3581,NaN,DE,Berlin,Berlin,Deutschland,[Berlin],NaN,5990c711a6fe4d3b69cff315
4,City Toilette,Public Bathroom,NaN,52.490667,13.397771,"[{'label': 'display', 'lat': 52.490667, 'lng':...",3191,NaN,DE,Berlin,Berlin,Deutschland,[Berlin],NaN,56cca7bfcd100886b2990361


In [298]:
print("Number of venues filtered", len(berlin_filtered.index))

Number of venues filtered 8


***Let's visualize the restrooms Foursquare found in Berlin***

In [297]:
# generate map centred around the city center of Berlin
venues_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add restrooms as yellow circle markers
for lat, lng, label in zip(berlin_filtered.lat, berlin_filtered.lng, berlin_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='yellow',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

***Next we want to see what Foursquare will find in Boston when we query for restrooms***

In [313]:
# Get the coordinates of Boston's center from Wikipedia : 42.358056, -71.063611
latitude = 42.358056
longitude = -71.063611

# Define query and search radius in meters
search_query = 'restroom'
radius = 30000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

# Send the GET Request and examine the results
results = requests.get(url).json()

# Get relevant part of JSON and transform it into a pandas dataframe
venues = results['response']['venues']

# transform venues into a dataframe
dataframe = json_normalize(venues)

print("Number of venues found", len(dataframe.index))

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

# extract restrooms from the list
boston_filtered = dataframe_filtered
boston_filtered.reset_index(drop=True, inplace=True)

print("Number of venues filtered", len(boston_filtered.index))

# generate map centred around the city center of Boston
venues_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add restrooms as yellow circle markers
for lat, lng, label in zip(boston_filtered.lat, boston_filtered.lng, boston_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='yellow',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Number of venues found 7
Number of venues filtered 7


<ipython-input-313-e5a7e6720e48>:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


***Quick Conclusion***

Boston's public restroom map provides the most professional and comprehensive view on public restrooms in this comparison. Next is the map generated in Folium using the data provided by Berlin's government. Foursquare has some room for improvement, as it shows only a fraction of the locations given by the two cities.